In [21]:
# create spark and sparkcontext objects
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import pandas as pd

from pyspark.sql.functions import to_timestamp
import time

import datetime
from pyspark.sql.functions import year, month, dayofmonth

In [2]:
train_df = spark.read.format("csv").option("header", "true").load("gender_age_train.csv")

In [3]:
test_df = spark.read.format("csv").option("header", "true").load("gender_age_test.csv")

In [4]:
events_df = spark.read.format("csv").option("header", "true").load("events.csv")

In [23]:
train_df.columns

['device_id', 'gender', 'age', 'group']

In [24]:
test_df.columns

['device_id']

In [5]:
# train_test=pd.concat([train_df,test_df])

# train_test=train_df.alias('a').join(test_df.alias('b'),fn.col('b.device_id')==fn.col('a.device_id')) 
# no same devices hence don't use

train_test_df=train_df

events_df=events_df.withColumnRenamed('device_id','tdevice_id')

train_test_events_df=train_test_df.alias('a').join(events_df.alias('b'),fn.col('b.tdevice_id')==fn.col('a.device_id'))
train_test_events_df=train_test_events_df.dropDuplicates()
train_test_events_df=train_test_events_df.drop('tdevice_id')
# train_test_events_df


In [49]:
train_test_events_df.dtypes

[('device_id', 'string'),
 ('gender', 'string'),
 ('age', 'string'),
 ('group', 'string'),
 ('event_id', 'string'),
 ('timestamp', 'string'),
 ('longitude', 'string'),
 ('latitude', 'string')]

In [6]:
train_test_events_df=train_test_events_df.withColumn('time-stamp',to_timestamp('timestamp')).drop('timestamp').drop('timestamp')


In [27]:
train_test_events_df.show()

+--------------------+------+---+------+--------+---------+--------+-------------------+
|           device_id|gender|age| group|event_id|longitude|latitude|         time-stamp|
+--------------------+------+---+------+--------+---------+--------+-------------------+
| 6721639827938905415|     F| 22|  F23-|     553|   118.21|   33.49|2016-05-04 00:03:04|
|-1053657946037918624|     F| 29|F29-32|     845|   120.53|   30.65|2016-05-06 00:45:20|
|-8345274263736281074|     F| 44|  F43+|    1407|     0.00|    0.00|2016-05-04 01:46:49|
|-6225748279665651076|     M| 44|  M39+|    1873|   120.46|   36.12|2016-05-04 02:12:13|
|-3762561587120285467|     M| 65|  M39+|    1999|   121.54|   31.29|2016-05-05 02:54:35|
| 3864672045456314600|     F| 29|F29-32|    2326|   113.19|   23.39|2016-05-04 03:08:04|
| 8794177577630781257|     F| 46|  F43+|    3108|     0.00|    0.00|2016-05-02 05:28:13|
| 8779095666317145799|     M| 33|M32-38|    3323|    82.90|   46.73|2016-05-05 05:22:04|
|-6376534734805845559

In [7]:
t=train_test_events_df.select('time-stamp')

In [8]:
start=time.clock()

In [33]:
# hours=dates.dt.hour

# fn.dayofmonth(train_test_events_df)

# t.select(
#     year('time-stamp'),
#     month('time-stamp'),
#     dayofmonth('time-stamp')
# ).show()

t=train_test_events_df.withColumn('year',year(fn.col('time-stamp'))).\
    withColumn('month',month(fn.col('time-stamp'))).\
    withColumn('day',dayofmonth(fn.col('time-stamp')))
t.show()

+--------------------+------+---+------+--------+---------+--------+-------------------+----+-----+---+
|           device_id|gender|age| group|event_id|longitude|latitude|         time-stamp|year|month|day|
+--------------------+------+---+------+--------+---------+--------+-------------------+----+-----+---+
| 6721639827938905415|     F| 22|  F23-|     553|   118.21|   33.49|2016-05-04 00:03:04|2016|    5|  4|
|-1053657946037918624|     F| 29|F29-32|     845|   120.53|   30.65|2016-05-06 00:45:20|2016|    5|  6|
|-8345274263736281074|     F| 44|  F43+|    1407|     0.00|    0.00|2016-05-04 01:46:49|2016|    5|  4|
|-6225748279665651076|     M| 44|  M39+|    1873|   120.46|   36.12|2016-05-04 02:12:13|2016|    5|  4|
|-3762561587120285467|     M| 65|  M39+|    1999|   121.54|   31.29|2016-05-05 02:54:35|2016|    5|  5|
| 3864672045456314600|     F| 29|F29-32|    2326|   113.19|   23.39|2016-05-04 03:08:04|2016|    5|  4|
| 8794177577630781257|     F| 46|  F43+|    3108|     0.00|    0

In [30]:
# p=train_test_events_df.join(t)

In [34]:
# p.show()